In [664]:
import uuid
import json
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict, Counter
import pandas as pd

In [665]:
def read_eval_results(results):
    with open(results, 'r') as json_file:
        json_list_str = list(json_file)
    json_data = dict()
    json_list = list()
    for json_str in json_list_str:
        result = json.loads(json_str)
        json_data[str(uuid.uuid4())] = result
        json_list.append(result)
    return json_list

### Configure file paths - run all cells for each file separately

In [764]:
FILE_PATH = "/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/1_0/Text generations/vllm_llama2_13b_1.0_repeat_copy_text.jsonl"
file_name = FILE_PATH.split('/')[-1]
BASE_PATH = '/'.join(FILE_PATH.split('/')[:-1])
GENERATION_PATH = BASE_PATH + "/" + file_name[:-6] + '_generation.pkl'
RESULT_JSONL_PATH = BASE_PATH + "/" + "RESULT_JSONLS_TEXT_1.0" + "/" + file_name[:-6] + '_result.jsonl'

In [765]:
GENERATION_PATH

'/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/1_0/Text generations/vllm_llama2_13b_1.0_repeat_copy_text_generation.pkl'

In [766]:
RESULT_JSONL_PATH

'/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/temperature_experiments_13b/1_0/Text generations/RESULT_JSONLS_TEXT_1.0/vllm_llama2_13b_1.0_repeat_copy_text_result.jsonl'

In [767]:
file_name.split('_')

['vllm', 'llama2', '13b', '1.0', 'repeat', 'copy', 'text.jsonl']

In [768]:
### DATASET EXTRACTION LOGIC ###

################################
## EXTRACTION LOGIC FOR GETTING THE DATASET
## EXTRACTION LOGIC FOR GETTING THE DATASET
def get_data(file_name):
    DATA = ''
    split_file_names = file_name.split('_')
    if len(split_file_names) == 7:
        DATA = ''.join(split_file_names[4:6])
    else:
        DATA = split_file_names[4]
    return DATA

In [769]:
DATASET = get_data(file_name)

In [770]:
DATASET

'repeatcopy'

In [771]:
############### LOAD DATA ############
data = read_eval_results(FILE_PATH)
len(data)

32

In [772]:
df = pd.DataFrame(data)

In [773]:
df.head()

,input,target,comment,generation
0,Repeat 5 times hello world,hello world hello world hello world hello worl...,LSTMs could solve this in 2014. Warmup example.,[[On repeating the words hello world five time...
1,repeat the word cat four times. After the seco...,cat cat meow cat cat,Let's make it more challenging with basic inde...,[[On repeating the word cat four times we get:...
2,"Repeat the word dog four times, but halfway th...",dog dog woof dog dog,Now let's add some basic inference on top of i...,[[On repeating the word dog four times we get:...
3,"Repeat all the world seven times, and after ev...",all the world all the world is a stage all the...,NaN,[[On repeating the word all the world seven ti...
4,"Say hungry three times, then hippo two times, ...",hungry hungry hungry hippo hippo feed me feed ...,NaN,[[On repeating the word hungry three times we ...


In [774]:
print(df['generation'][2][0][3].split('\n\n')[0])

On repeating the word dog four times we get: dog dog dog dog
Halfway through if we say woof, we have to say woof in between the 2nd word and the 3rd word
The answer is: dog dog woof dog dog


In [775]:
all_generations = []
for generations in df['generation']:
    generations = generations[0]
    processed_generations = []
    for generation in generations:
        processed_generations.append(generation.split('\n\n')[0])
    all_generations.append(processed_generations)

In [776]:
len(all_generations)

32

In [777]:
for generations in all_generations:
    for generation in generations:
        assert generation == generation.split('\n\n')[0]

### Write processed generations to file

In [778]:
import pickle
with open(GENERATION_PATH, "wb") as f:
    pickle.dump(all_generations, f)

In [1]:
########### FUNCTIONS FOR DATE AND NUMBER EXTRACTION ########################

import re

def extract_number(string):
    pattern = r'answer is:? (\d+)'
    match = re.search(pattern, string)
    if match:
        return int(match.group(1))
    else:
        return None
    
def extract_date(string):
    pattern = r"answer is:? (\d{2}/\d{2}/\d{4})"
    match = re.search(pattern, string)
    if match:
        return match.group(1)
    else:
        return None
#[r"The answer is\s*:\s*[^0-9]*?(\d+)[^0-9]*", r"The answer is\s+[^0-9]*?(\d+)[^0-9]*"],

REGEX_NUMERIC = r'\d+(?:\.\d+)?'
REGEX_DATE = r'\d{2}/\d{2}/\d{4}'

In [780]:
def better_extraction(generations, regex_string, extract_answer_is):
    candidate_ans =[]
    for i in generations:
        i = i.split('\n\n')[0]
        i = i.replace('"""', '')
        res = extract_answer_is(i)
        print ("Res found as the answer is", res)
        if res == None:
            print("Line of interest: ", i.split("\n")[-1])
            match = re.findall(regex_string, i.split("\n")[-1])
            if len(match) > 0:
                last_number_str = match[-1]
                try:
                    last_number = float(last_number_str)
                    res = float(last_number)
                except Exception as e:
                    res = last_number_str
                print ("Res found as last num", res)
                candidate_ans.append(res)
            else:
                res = ""
                print (i)
                print("still not found")
        else:
            try:
                candidate_ans.append(float(res))
            except:
                candidate_ans.append(res)
        print("****************************************")
        
    return candidate_ans

In [781]:
# df['generation'][0]

In [782]:
all_candidates_ans = []
for index,row in df.iterrows():
    if DATASET == "dateunderstanding":
        all_candidates_ans.append(better_extraction(row['generation'][0], REGEX_DATE, extract_date))
    else:
        all_candidates_ans.append(better_extraction(row['generation'][0], REGEX_NUMERIC, extract_number))

Res found as the answer is None
Line of interest:  The answer is: hello world hello world hello world hello world hello world
On repeating the words hello world five times we get: hello world hello world hello world hello world hello world
The answer is: hello world hello world hello world hello world hello world
still not found
****************************************
Res found as the answer is None
Line of interest:  The answer is: hello world hello world hello world hello world hello world
On repeating the words hello world five times we get: hello world hello world hello world hello world hello world
The answer is: hello world hello world hello world hello world hello world
still not found
****************************************
Res found as the answer is None
Line of interest:  hello world
On repeating the word hello world 5 times we get: hello world hello world hello world hello world
The answer is:
hello world
still not found
****************************************
Res found a

In [783]:
len(all_candidates_ans)

32

In [784]:
# all_candidates_ans

In [745]:
def pad_with_non_extractable(results):
    new_results = []
    for result in results:
#         if len(result) < 10:
        pad_no = 10 - len(result)
        new_result = result + ["non_extractable"]*pad_no
        new_results.append(new_result)
    return new_results

In [746]:
padded_all_candidates_ans = pad_with_non_extractable(all_candidates_ans)

In [747]:
padded_all_candidates_ans

[[14.0, 13.0, 13.0, 12.0, 11.0, 13.0, 15.0, 11.0, 14.0, 8.0],
 [14.0, 14.0, 16.0, 16.0, 17.0, 15.0, 16.0, 14.0, 14.0, 'non_extractable'],
 [3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0],
 [11.0, 12.0, 14.0, 14.0, 12.0, 12.0, 14.0, 16.0, 14.0, 13.0],
 [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0],
 [1.0, 3.0, 3.0, 3.0, 3.0, 2.0, 3.0, 3.0, 3.0, 3.0],
 [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
 [11.0, 20.0, 15.0, 15.0, 7.0, 14.0, 12.0, 3.0, 7.0, 'non_extractable'],
 [7.0, 11.0, 8.0, 10.0, 8.0, 9.0, 8.0, 8.0, 11.0, 8.0],
 [10.0, 10.0, 9.0, 9.0, 15.0, 10.0, 12.0, 10.0, 11.0, 10.0],
 [12.0, 10.0, 10.0, 9.0, 9.0, 10.0, 11.0, 6.0, 11.0, 13.0],
 [11.0, 11.0, 12.0, 12.0, 7.0, 12.0, 11.0, 6.0, 11.0, 11.0],
 [6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0],
 [9.0, 9.0, 10.0, 9.0, 9.0, 7.0, 9.0, 8.0, 7.0, 9.0],
 [10.0, 10.0, 11.0, 11.0, 10.0, 11.0, 10.0, 11.0, 11.0, 10.0],
 [7.0, 6.0, 8.0, 5.0, 3.0, 5.0, 4.0, 9.0, 0.0, 'non_extractable'],
 [10.0, 11.0, 15.0, 10.0, 11.0, 11.0, 1

In [748]:
str_padded_all_candidates_ans = []
for answers in padded_all_candidates_ans:
    str_answers = []
    for ans in answers:
        str_answers.append(str(ans))
    str_padded_all_candidates_ans.append(str_answers)

In [749]:
non_extractable_no = 0
for generations in str_padded_all_candidates_ans:
    non_extractable_no += len([result for result in generations if result == 'non_extractable'])

In [750]:
non_extractable_no/(len(str_padded_all_candidates_ans)*10)

0.0188

In [751]:
str_padded_all_candidates_ans

[['14.0',
  '13.0',
  '13.0',
  '12.0',
  '11.0',
  '13.0',
  '15.0',
  '11.0',
  '14.0',
  '8.0'],
 ['14.0',
  '14.0',
  '16.0',
  '16.0',
  '17.0',
  '15.0',
  '16.0',
  '14.0',
  '14.0',
  'non_extractable'],
 ['3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0', '3.0'],
 ['11.0',
  '12.0',
  '14.0',
  '14.0',
  '12.0',
  '12.0',
  '14.0',
  '16.0',
  '14.0',
  '13.0'],
 ['5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '5.0', '4.0'],
 ['1.0', '3.0', '3.0', '3.0', '3.0', '2.0', '3.0', '3.0', '3.0', '3.0'],
 ['2.0', '2.0', '2.0', '2.0', '2.0', '2.0', '2.0', '2.0', '2.0', '2.0'],
 ['11.0',
  '20.0',
  '15.0',
  '15.0',
  '7.0',
  '14.0',
  '12.0',
  '3.0',
  '7.0',
  'non_extractable'],
 ['7.0', '11.0', '8.0', '10.0', '8.0', '9.0', '8.0', '8.0', '11.0', '8.0'],
 ['10.0',
  '10.0',
  '9.0',
  '9.0',
  '15.0',
  '10.0',
  '12.0',
  '10.0',
  '11.0',
  '10.0'],
 ['12.0', '10.0', '10.0', '9.0', '9.0', '10.0', '11.0', '6.0', '11.0', '13.0'],
 ['11.0',
  '11.0',
  '12.0',
 

In [752]:
from collections import Counter


def get_calibration_dict_and_majority_answer(candidates):

    max_a=[] #Maximum occuring value in list ---- answer
    cal_all=[] # Calibration dictionary

    for l in candidates:
        unique, counts = np.unique(np.array(l), return_counts=True)
        d = dict(np.asarray((unique, counts / len(l))).T)
        cal_all.append(d)
        key_counts = Counter(l)

        # Find the key with the highest count
        try:
            most_common_key = key_counts.most_common(1)[0][0]
        except:
            most_common_key = None
        max_a.append(most_common_key)
    return cal_all, max_a

### Accuracy calculation

In [753]:
###### To calculate accuracy ############
def accuracy(list1, list2):
    matches =0
    for i in range(len(list1)):
        if list1[i] == list2[i]:
            matches += 1

# Calculate the accuracy as a percentage
    accuracy = (matches / len(list1)) * 100

    return accuracy# prints 60.0

In [754]:
cal_all, max_a = get_calibration_dict_and_majority_answer(all_candidates_ans)

In [755]:
target = df['target'].tolist()
if DATASET in ["gsm", "gsmhardv2", "objectcounting"]:
# target = [str(float(i)) for i in target]
    target = [float(i) for i in target]
# target = df['answer'].tolist()

In [756]:
# np.mean(np.array(max_a) == np.array(target))

In [757]:
print(accuracy(target, max_a))

57.599999999999994


### Calibration results 

In [758]:
########## To calculate calibration use cal_all list ################

In [759]:
cal_all, max_a = get_calibration_dict_and_majority_answer(str_padded_all_candidates_ans)

In [760]:
target = df['target'].tolist()
if DATASET in ["gsm", "gsmhardv2", "objectcounting"]:
    target = [str(float(i)) for i in target]
#     target = [float(i) for i in target]
# target = df['answer'].tolist()

In [761]:
result_jsonl = []
for calibration_result,answer, target in zip(cal_all, max_a, target):
    json_result = dict()
    json_result['calibration_results'] = [calibration_result]
    json_result['answer'] = answer
    json_result['target'] = target
    json_result['score'] = 1 if answer == target else 0
    result_jsonl.append(json_result)

In [762]:
result_jsonl

[{'calibration_results': [{'11.0': '0.2',
    '12.0': '0.1',
    '13.0': '0.3',
    '14.0': '0.2',
    '15.0': '0.1',
    '8.0': '0.1'}],
  'answer': '13.0',
  'target': '8.0',
  'score': 0},
 {'calibration_results': [{'14.0': '0.4',
    '15.0': '0.1',
    '16.0': '0.3',
    '17.0': '0.1',
    'non_extractable': '0.1'}],
  'answer': '14.0',
  'target': '15.0',
  'score': 0},
 {'calibration_results': [{'3.0': '1.0'}],
  'answer': '3.0',
  'target': '3.0',
  'score': 1},
 {'calibration_results': [{'11.0': '0.1',
    '12.0': '0.3',
    '13.0': '0.1',
    '14.0': '0.4',
    '16.0': '0.1'}],
  'answer': '14.0',
  'target': '14.0',
  'score': 1},
 {'calibration_results': [{'4.0': '0.1', '5.0': '0.9'}],
  'answer': '5.0',
  'target': '5.0',
  'score': 1},
 {'calibration_results': [{'1.0': '0.1', '2.0': '0.1', '3.0': '0.8'}],
  'answer': '3.0',
  'target': '3.0',
  'score': 1},
 {'calibration_results': [{'2.0': '1.0'}],
  'answer': '2.0',
  'target': '2.0',
  'score': 1},
 {'calibration_result

In [763]:
# import json
# with open("/Users/sankethrangreji/Coursework/11797 Question Answering/QA11797/LLaMA-2 70b results/llama_gsm_text.jsonl", "w") as f:
#     # iterate over list of dictionaries
#     for d in result_jsonl:
#         # convert dictionary to JSON string
#         json_str = json.dumps(d)
#         # write JSON string to file followed by newline character
#         f.write(json_str + "\n")
import json
with open(RESULT_JSONL_PATH, "w") as f:
    json_obj = json.dumps(result_jsonl)
    f.write(json_obj)